In [1]:

# def create_and_set_working_directory(project_folder):
#   if os.path.isdir(root_dir + project_folder) == False:
#     os.mkdir(root_dir + project_folder)
#     print(root_dir + project_folder + ' did not exist but was created.')


#   os.chdir(root_dir + project_folder)

#   print('\nYour working directory was changed to ' + root_dir + project_folder)


# create_and_set_working_directory(project_folder)
from huggingface_hub import notebook_login

notebook_login()


In [2]:
! git clone https://github.com/hyintell/BLOOM-fine-tuning.git
%cd BLOOM-fine-tuning
! pip install -r requirements.txt

Cloning into 'BLOOM-fine-tuning'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 16 (delta 2), reused 13 (delta 2), pack-reused 0
Receiving objects: 100% (16/16), 4.16 KiB | 852.00 KiB/s, done.
Resolving deltas: 100% (2/2), done.
/kaggle/working/BLOOM-fine-tuning
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 48.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.

# Libraries

In [ ]:
#!pip install transformers

In [3]:
import torch
import transformers
from transformers import BloomTokenizerFast, BloomForCausalLM, TrainingArguments, pipeline, AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling

from datasets import load_dataset

from utils import ModifiedTrainer, tokenise_data, data_collator

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


 # Main

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [6]:
# tokeniser = AutoTokenizer.from_pretrained('bigcode/tiny_starcoder_py')
# model = AutoModelForCausalLM.from_pretrained('bigcode/tiny_starcoder_py')
#AutoModelForSeq2SeqLM
model_name = "bloom-1b7"
model = BloomForCausalLM.from_pretrained(f"bigscience/{model_name}").to(device)
tokeniser = BloomTokenizerFast.from_pretrained(f"bigscience/{model_name}", add_prefix_space=True)

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(f"bigscience/{model_name}", add_prefix_space=True)

In [7]:
from datasets import load_dataset

dataset = load_dataset("phongmt184172/python_data_27k")
input_ids = tokenise_data(dataset, tokeniser)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/19056 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4084 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/4084 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/phongmt184172___parquet/phongmt184172--python_data_27k-20793de6bb11d86b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 19056/19056 [00:04<00:00, 3910.03it/s]


In [8]:
print(dataset['train'][0])

{'id': 14605, 'text': 'Write a program that raises a custom exception and logs the exception details to the console.', 'label': 'Certainly! Here\'s an example program that raises a custom exception and logs the exception details to the console:\n\n```python\nimport logging\n\nclass CustomException(Exception):\n    pass\n\ndef raise_custom_exception():\n    """\n    This function raises a custom exception and logs the exception details to the console.\n\n    :raises CustomException: When the custom condition is met.\n    """\n    try:\n        # Custom condition to raise the exception\n        number = int(input("Enter a number greater than 10: "))\n        if number <= 10:\n            raise CustomException("Number is not greater than 10.")\n    except ValueError:\n        logging.exception("Invalid input. Please enter a valid integer.")\n    except CustomException as ce:\n        logging.exception("Custom exception raised: {}".format(str(ce)))\n    except Exception:\n        logging.e

In [9]:
# def format_instruction(sample):
#         return f"""### Instruction:
#     Use the Input given to write the Response, which is a programming code that can solve the following Input:

#     ### Input:
#     {sample['text']}

#     ### Response:
#     {sample['label']}
#     """

In [10]:
!pip uninstall peft
!pip install git+https://github.com/huggingface/peft.git

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-tu04pq91
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-tu04pq91
  Resolved https://github.com/huggingface/peft.git to commit dbd40d96a15d9b8b04c3582bb9ea00ae24f56348
  Installing build dependencies ... done
  Get

In [11]:
from peft import (
    get_peft_model,
    PromptTuningConfig
)

prompt_config = PromptTuningConfig(
    task_type="CAUSAL_LM",
    peft_type="P_TUNING",
    num_virtual_tokens=20
)

In [12]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# model_name = "google/flan-t5-xl"
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.enable_input_require_grads()
prompt_model = get_peft_model(model, prompt_config)
prompt_model.print_trainable_parameters()
prompt_model

trainable params: 40,960 || all params: 1,722,449,920 || trainable%: 0.0023780081803481405


PeftModelForCausalLM(
  (base_model): BloomForCausalLM(
    (transformer): BloomModel(
      (word_embeddings): Embedding(250880, 2048)
      (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (h): ModuleList(
        (0-23): 24 x BloomBlock(
          (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (self_attention): BloomAttention(
            (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
            (dense): Linear(in_features=2048, out_features=2048, bias=True)
            (attention_dropout): Dropout(p=0.0, inplace=False)
          )
          (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (mlp): BloomMLP(
            (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
            (gelu_impl): BloomGelu()
            (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
          )
        )
      

In [13]:
prompt_model.to(device)


PeftModelForCausalLM(
  (base_model): BloomForCausalLM(
    (transformer): BloomModel(
      (word_embeddings): Embedding(250880, 2048)
      (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (h): ModuleList(
        (0-23): 24 x BloomBlock(
          (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (self_attention): BloomAttention(
            (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
            (dense): Linear(in_features=2048, out_features=2048, bias=True)
            (attention_dropout): Dropout(p=0.0, inplace=False)
          )
          (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (mlp): BloomMLP(
            (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
            (gelu_impl): BloomGelu()
            (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
          )
        )
      

In [14]:
# !pip install trl
# from trl import SFTTrainer

In [15]:
prompt_model.gradient_checkpointing_enable()
prompt_model.is_parallelizable = True
prompt_model.model_parallel = True
import os
os.environ['WANDB_DISABLED'] = 'true'

training_args = TrainingArguments(
    "output",
    fp16=True,
    gradient_accumulation_steps= 1,
    per_device_train_batch_size = 2,
    learning_rate = 2e-5,
    num_train_epochs=2,
    logging_steps=10,
    save_total_limit=1
)

trainer = ModifiedTrainer(
    model=prompt_model,
    train_dataset=input_ids,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokeniser, mlm=False),
    #formatting_func = format_instruction,
)

trainer.train()

prompt_model.save_pretrained("prompt-tune-peft")


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
10,12.140100
20,12.096300
30,12.000700
40,11.927300
50,12.062600
60,12.008200
70,12.019400
80,12.080700
90,11.943000
100,11.990100


In [16]:
peft_model_id = "sonnguyen25/bloom-1b7_PROMPT_TUNING_CAUSAL_LM"
model.push_to_hub("sonnguyen25/bloom-1b7_PROMPT_TUNING_CAUSAL_LM", use_auth_token=True)

pytorch_model.bin:   0%|          | 0.00/6.89G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sonnguyen25/bloom-1b7_PROMPT_TUNING_CAUSAL_LM/commit/5013694f1efffd336eb0bce4d271cf7592ed5db2', commit_message='Upload BloomForCausalLM', commit_description='', oid='5013694f1efffd336eb0bce4d271cf7592ed5db2', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
peft_model_id = "sonnguyen25/bloom-1b7_PROMPT_TUNING_CAUSAL_LM_2"
prompt_model.push_to_hub("sonnguyen25/bloom-1b7_PROMPT_TUNING_CAUSAL_LM_2", use_auth_token=True)

adapter_model.bin:   0%|          | 0.00/165k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sonnguyen25/bloom-1b7_PROMPT_TUNING_CAUSAL_LM_2/commit/44639c0cb3a95fac5e2adc61dda6c8ed6077ff80', commit_message='Upload model', commit_description='', oid='44639c0cb3a95fac5e2adc61dda6c8ed6077ff80', pr_url=None, pr_revision=None, pr_num=None)